In [7]:
import pygame
import numpy as np
import cv2
import time
from handdetector import HandDetector
import scipy

1. **Audio Parameters:**
   - sample_rate: Defines the number of audio samples per second, set to 44,100 Hz, which is a standard sample rate for high-quality audio.
   - buffer_size: Determines the duration of audio waveform chunks in seconds, set to 0.01 seconds. This controls the granularity of the sound processing.

2. **Sound Envelope Parameters:**
   - sound_params: A dictionary containing the parameters for shaping the sound envelope:
     -attack: The time it takes for the sound to reach its peak after the note is played, set to 0.01 seconds (short attack).
     - decay: The time it takes for the sound to decrease from the peak to the sustain level, set to 0.3 seconds.
     - sustain: The level at which the sound remains after decay, set to 0.7 (moderate sustain).
     - release: The time it takes for the sound to fade to silence after the note is released, set to 0.2 seconds.

3. **Frequency Calculation:**
   - base_frequency: Sets the frequency of the note 'do' to 800 Hz.
   - semitone_ratio: The mathematical ratio between the frequencies of two adjacent semitones, calculated using the 12th root of 2.
   - frequencies: A dictionary mapping each musical note ('do', 're', 'mi', etc.) to its corresponding frequency. The frequencies for notes are calculated relative to 'do' by multiplying the base frequency with the appropriate power of semitone_ratio.

4. **Finger to Note Mapping:**
   - finger_to_note: A dictionary mapping specific hand landmarks (finger positions identified by their index) to musical notes. These mappings are used to play corresponding notes when a specific finger position is detected by the hand-tracking system.

In summary, this code sets up the necessary parameters to generate and play musical notes with specific characteristics (envelope shaping, frequency calculation) in response to hand gestures detected by a vision system.

In [8]:
# Parameters

sample_rate = 44100 
buffer_size = 0.01  


sound_params = {
    'attack': 0.01,        
    'decay': 0.3,          
    'sustain': 0.7,        
    'release': 0.2,        
}

base_frequency = 800  # Frequency of 'do' 
semitone_ratio = 2 ** (1 / 12)  # Ratio of frequencies for each semitone

# Calculate frequencies for other notes relative to 'do'
frequencies = {
    'do': base_frequency,
    're': base_frequency * semitone_ratio ** 2,  # D4
    'mi': base_frequency * semitone_ratio ** 4,  # E4
    'fa': base_frequency * semitone_ratio ** 5,  # F4
    'sol': base_frequency * semitone_ratio ** 7,  # G4
    'la': base_frequency * semitone_ratio ** 9,  # A4
    'si': base_frequency * semitone_ratio ** 11  # B4
}

# Finger landmarks to keys mapping
finger_to_note = {
    21: 'do', 5: 're', 8: 'mi', 12: 'fa', 16: 'sol', 20: 'la', 17: 'si'
}

In [9]:
def generate_piano_waveform(frequency, duration, sample_rate):
    num_samples = int(sample_rate * duration)
    t = np.linspace(0, duration, num_samples, endpoint=False)
    
    # Piano-like harmonics
    harmonics = [1, 2, 3, 4, 5]
    amplitudes = [1.0, 0.5, 0.3, 0.2, 0.1]
    
    waveform = np.zeros_like(t)
    for harmonic, amplitude in zip(harmonics, amplitudes):
        waveform += amplitude * np.sin(2 * np.pi * frequency * harmonic * t)
    
    # Apply envelope using sound_params
    attack = int(num_samples * sound_params['attack'])
    decay = int(num_samples * sound_params['decay'])
    sustain_level = sound_params['sustain']
    
    if attack + decay > num_samples:
        decay = num_samples - attack
    
    release = num_samples - attack - decay
    
    envelope = np.zeros_like(t)
    envelope[:attack] = np.linspace(0, 1, attack)
    envelope[attack:attack+decay] = np.linspace(1, sustain_level, decay)
    envelope[attack+decay:] = np.linspace(sustain_level, 0, release)
    
    waveform *= envelope
    
    # Normalize
    waveform /= np.max(np.abs(waveform))
    
    # High-pass filter to remove low-frequency noise
    cutoff_freq = 80  # Cutoff frequency in Hz
    b, a = scipy.signal.butter(1, cutoff_freq / (0.5 * sample_rate), btype='high')
    waveform = scipy.signal.filtfilt(b, a, waveform)
    
    # Create stereo waveform
    stereo_waveform = np.array([waveform, waveform]).T.copy()
    return (stereo_waveform * 0.5).astype(np.float32)


In [10]:
def waveform_to_sound(waveform):
    waveform=(waveform*32767).astype(np.int16)
    sound=pygame.sndarray.make_sound(waveform)
    return sound

In [11]:
def distance(lmList, epsilon):
    thumb=np.array(lmList[4])
    notes_distances={

    }
    
    for finger, note in finger_to_note.items():
        dist=np.linalg.norm(np.array(lmList[finger])-thumb)
        notes_distances[note]=dist

    notes_distance={note:0 for note in finger_to_note.values()}
    
    min_note=min(notes_distances,key=notes_distances.get)
    
    if notes_distances[min_note]<=epsilon:
        notes_distance[min_note]=1

    return notes_distance

In [12]:
def draw_piano(window, notes_on):
    key_width = width // len(frequencies)
    for i, (note, freq) in enumerate(frequencies.items()):
        color = (255, 0, 0) if notes_on.get(note, False) else (255, 255, 255)
        pygame.draw.rect(window, color, (i * key_width, 0, key_width, height))
        pygame.draw.rect(window, (0, 0, 0), (i * key_width, 0, key_width, height), 2)
        font = pygame.font.Font(None, 36)
        text = font.render(note, True, (0, 0, 0))
        window.blit(text, (i * key_width + 10, height - 40))

In [13]:
#the main function

def main():
    pTime=0
    cap=cv2.VideoCapture(0)
    detector=HandDetector()
    
    while True:
        success,img=cap.read()
        if not success:
            break
        
        img=cv2.flip(img,1)
        img=detector.findHands(img,True)
        lmList=detector.findPosition(img,0,True)
        
        notes_on={}
        if lmList:
            extracted_columns=[[row[1],row[2]] for row in lmList]
            notes_on=distance(extracted_columns,30)
            
            for note, is_playing in notes_on.items():
                if is_playing:
                    waveform=generate_piano_waveform(frequencies[note],buffer_size,sample_rate)
                    sound=waveform_to_sound(waveform)
                    sound_channels[note].play(sound,loops=-1)
                else:
                    sound_channels[note].fadeout(100) 
        
        cTime=time.time()
        fps=1/(cTime-pTime)
        pTime=cTime
        
        cv2.putText(img,f"FPS:{int(fps)}",(10,70),cv2.FONT_HERSHEY_PLAIN,3,(0,255,255),3)
        cv2.imshow("Virtual Piano",img)
        
        #window.fill((200, 200, 200))
        #draw_piano(window, notes_on)
        #pygame.display.flip()
        
        if cv2.waitKey(1) and 0xFF==ord('q'):
            break
    
    cap.release()
    cv2.destroyAllWindows()
    pygame.quit()

In [14]:
# Initialize Pygame
pygame.init()
pygame.mixer.init(frequency=sample_rate, size=-16, channels=2)

"""# Initialize Pygame display
width, height = 700, 300
window = pygame.display.set_mode((width, height))
pygame.display.set_caption("Virtual Piano with Vision")

# Generate sound channels
sound_channels = {note: pygame.mixer.Channel(i) for i, note in enumerate(frequencies)}"""

if __name__ == "__main__":
    main()


NameError: name 'pygame' is not defined